In [70]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
import pandas as pd
import shutil
from pathlib import Path
from config.config import OBSERVACIONS_FILTRAT_DIR, OBSERVACIONS_DIR, ESTACIONS_DIR
import numpy as np

In [72]:
#if path exists, delete it
try:
    shutil.rmtree(OBSERVACIONS_FILTRAT_DIR)
except:
    pass

OBSERVACIONS_FILTRAT_DIR.mkdir()

In [73]:
"""
De totes les estacions d'aforament, les que definim a continuació són les que en gran part tenen dades que semblen correctes.
La resta, les descartem.
Definit per Vicenç i Laia
"""

besos  = [19,20,30,35,38,39,40,50,51,59,62]
fluvia = [18,42,69]
llobregat = [1,3,4,7,9,11,12,17,23,24,25,26,29,31,41,45,46,49,54,57,58,71,72]
muga = [5,8,36,43,44,55,61]
sud = [13,16,37,66,70]
ter = [21,22,28,33,34,47,48,52,53,60,65,67,68,73]

estacions_incloses = set(besos + fluvia + llobregat + muga + sud + ter)

In [74]:
#Min and max date in all the datasets in observacions_dir
min_date = min(map(lambda path: pd.read_csv(path)["Date"].min(), OBSERVACIONS_DIR.glob("*.csv")))
max_date = max(map(lambda path: pd.read_csv(path)["Date"].max(), OBSERVACIONS_DIR.glob("*.csv")))

print(f"Min date: {min_date}")
print(f"Max date: {max_date}")

Min date: 2001-01-01
Max date: 2021-02-16


In [75]:
#Copy all files from observations to observations_filtrat
for file in OBSERVACIONS_DIR.glob('*.csv'):

    df = pd.read_csv(file)
    
    #Numero d'estacio
    name_file = file.name
    estacio = int(name_file.replace('.csv', '').replace('a', ''))

    if estacio not in estacions_incloses:
        continue

    #date column to datetime
    df['Date'] = pd.to_datetime(df['Date'])

    #Besòs
    if estacio == 19:
        df = df[(df['Date'] <= '2004-04-15') | (df['Date'] >= '2006-04-19')]

    #Llobregat
    elif estacio == 4:
        df = df[(df['Date'] >= '2016-01-01')]
    elif estacio == 12:
        df = df[(df['Date'] <= '2012-08-19') | (df['Date'] >= '2015-01-01')]
    elif estacio == 46:
        df = df[((df['Date'] <= '2015-12-01') | (df['Date'] >= '2016-01-01')) & ((df['Date'] <= '2017-03-30') | (df['Date'] >= '2017-04-12'))]
    
    #Muga
    elif estacio == 36:
        df = df[(df['Date'] <= '2009-08-09') | (df['Date'] >= '2009-08-18')]
    
    #Ter
    elif estacio == 48:
        df = df[((df['Date'] <= '2006-01-22') | (df['Date'] >= '2008-02-27')) & (df['Date'] <= '2018-11-30')]
    elif estacio == 60:
        df = df[(df['Date'] <= '2015-12-31') | (df['Date'] >= '2019-10-22')]
    elif estacio == 65:
        df = df[((df['Date'] <= '2017-05-15') | (df['Date'] >= '2017-07-05')) & ((df['Date'] <= '2018-11-30') | (df['Date'] >= '2019-07-01'))]
    elif estacio == 73:
        df = df[df['Date'] != '2020-09-30']


    
    # create a new DataFrame with all dates between min_date and max_date
    all_dates = pd.DataFrame({'Date': pd.date_range(start=pd.to_datetime(min_date), end=pd.to_datetime(max_date))})

    # merge the two DataFrames using a left join
    merged_df = pd.merge(all_dates, df, on='Date', how='left')

    # replace missing flow values with NaN
    merged_df['Flow'] = merged_df['Flow'].fillna(np.nan)
    

    merged_df.to_csv(Path(OBSERVACIONS_FILTRAT_DIR, name_file), index=False)